In [1]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import time
import torch
import json
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer

# Track model loading time
start_load = time.time()
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
load_time = time.time() - start_load

def get_memory_usage():
    return torch.cuda.memory_allocated() / 1e6  # Convert bytes to MB

def run_inference_hft(prompts, token_sizes, num_runs=5):
    results = []
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        for tokens in token_sizes:
            latencies = []
            memory_usage = []
            throughput_vals = []

            for _ in range(num_runs):
                gc.collect()
                torch.cuda.empty_cache()
                start_mem = get_memory_usage()

                start_time = time.time()
                outputs = model.generate(**inputs, max_new_tokens=tokens)
                end_time = time.time()

                peak_mem = get_memory_usage() - start_mem
                inference_time = end_time - start_time
                tokens_generated = tokens

                latencies.append(inference_time)
                memory_usage.append(peak_mem)
                throughput_vals.append(tokens_generated / inference_time)

            results.append({
                "method": "HFT",
                "tokens": tokens,
                "latency_mean": sum(latencies) / num_runs,
                "latency_std": torch.std(torch.tensor(latencies)).item(),
                "memory_usage_mean": sum(memory_usage) / num_runs,
                "throughput_mean": sum(throughput_vals) / num_runs,
                "batch_size": len(prompts),
                "model_load_time": load_time
            })

    return results

# Run inference
prompts = ["Explain in detail about the string theory"]
token_sizes = [10, 50, 100, 200, 500, 1000, 1500]
results_hft = run_inference_hft(prompts, token_sizes)

# Save results
with open("hft_results.json", "w") as f:
    json.dump(results_hft, f, indent=4)

print("HFT Inference Completed.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

HFT Inference Completed.
